In [1]:
import pandas as pd
import re
import numpy as np
import time
import datetime
from utils import review_df_pr

In [60]:
review_1 = pd.read_csv("1.csv")
review_2 = pd.read_csv("2.csv")
review_3 = pd.read_csv("3.csv")
review_4 = pd.read_csv("4.csv")

na_index = list(review_1[review_1['timestamp'].isna()].index)
for i in na_index:
    tmp = review_1.iloc[(i)][['itemid', 'userid']]
    review_1.loc[(i-1),'comment'] = review_1.loc[(i-1)]['comment'] + '\n' + tmp[0]
    review_1.loc[(i-1),'option'] = tmp[1]

review_1.dropna(axis=0, subset=['timestamp'], inplace=True)
review_1.dropna(axis=0, subset=['comment'], inplace=True)

na_index = list(review_3[review_3['timestamp'].isna()].index)
i = na_index[0]
tmp = review_3.iloc[(i)][['Unnamed: 0', 'itemid']]
review_3.loc[(i-1),'comment'] = review_3.loc[(i-1)]['comment'] + '\n' + tmp[0]
review_3.loc[(i-1),'option'] = tmp[1]

i = na_index[1]
tmp = review_3.iloc[i:(i+2),:][['Unnamed: 0', 'itemid']]
review_3.loc[(i-1),'comment'] = review_3.loc[(i-1)]['comment'] + '\n' + tmp.iloc[0][0] + '\n' + tmp.iloc[1][0]
review_3.loc[(i-1),'option'] = tmp.iloc[1][1]

review_1.dropna(subset='comment', inplace=True)
review_1['comment'] = review_1['comment'].apply(lambda x: re.sub(r'\n',' ',x))

review_2.dropna(subset='review', inplace=True)
review_2['review'] = review_2['review'].apply(lambda x: re.sub(r'\n',' ',x))

review_3.dropna(subset='comment', inplace=True)
review_3['comment'] = review_3['comment'].apply(lambda x: re.sub(r'\n',' ',x))

review_4.dropna(subset='comment', inplace=True)
review_4['comment'] = review_4['comment'].apply(lambda x: re.sub(r'\n',' ',x))

review_3.dropna(axis=0, subset=['userid'], inplace=True)
review_3.dropna(axis=0, subset=['comment'], inplace=True)
review_2.dropna(axis=0, subset=['review'], inplace=True)
review_4.dropna(axis=0, subset=['comment'], inplace=True)
review_3.drop('Unnamed: 0', axis=1, inplace=True)

review_2.columns = review_1.columns
review_total = pd.concat([review_1, review_2, review_3, review_4], axis=0)
review_total.columns = ['item_id', 'user_id', 'time', 'star_avg', 'star_durability', 'star_design', 'star_cost', 'star_delivery', 'review', 'option']
review_total.reset_index(drop=True, inplace=True)

review_total['item_id'] = review_total['item_id'].apply(int)
review_total['user_id'] = review_total['user_id'].apply(int)
review_total['year'] = review_total['time'].apply(lambda x: int(x[:4]))
review_total['month'] = review_total['time'].apply(lambda x: int(x[5:7]))
review_total['day'] = review_total['time'].apply(lambda x: int(x[8:]))
review_total['review'] = review_total['review'].apply(lambda x: re.sub(r'\n',' ',x))

condition = (review_total['star_avg']>0) & (review_total['star_durability']==0) & (review_total['star_design']==0) & (review_total['star_cost']==0) & (review_total['star_delivery']==0)
idxs = review_total.loc[condition].index
review_total.iloc[idxs, 4] = review_total.iloc[idxs, 3]
review_total.iloc[idxs, 5] = review_total.iloc[idxs, 3]
review_total.iloc[idxs, 6] = review_total.iloc[idxs, 3]
review_total.iloc[idxs, 7] = review_total.iloc[idxs, 3]
review_total.loc[review_total['option'].isna(), ['option']] = 'NO'
review_total.drop_duplicates(inplace=True)

/tmp/ipykernel_81706/1988394935.py:1: DtypeWarning: Columns (0,1) have mixed types. Specify dtype option on import or set low_memory=False.
  review_1 = pd.read_csv("1.csv")
/tmp/ipykernel_81706/1988394935.py:3: DtypeWarning: Columns (0,1) have mixed types. Specify dtype option on import or set low_memory=False.
  review_3 = pd.read_csv("3.csv")


In [64]:
review_total.to_csv("review_total.csv", index=False)

# inter

In [2]:
# tmp = pd.read_csv("review_total.csv", lineterminator='\n')
tmp = pd.read_csv("cut_user_item.csv", lineterminator='\n')
inter_recbole = tmp.copy()
inter_recbole['time'] = inter_recbole['time'].apply(lambda x: int(time.mktime(datetime.datetime.strptime(x, "%Y.%m.%d").timetuple())))
inter_recbole.head(3)

,item_id,user_id,time,star_avg,star_durability,star_design,star_cost,star_delivery,review,option,year,month,day
0,321021,367873,1598572800,5.00,5.0,5.0,5.0,5.0,첫 마디~~ 너무 예뻐요 깜직한 디자인에 거름망도 아주 정교하게 촘촘해서 차 마실...,단품,2020,8,28
1,321021,13531239,1629676800,4.25,5.0,5.0,3.0,4.0,아담하고 찾던 사이즈라 좋아요~ 생각보다 튼튼해 보입니다,단품,2021,8,23
2,321021,3010490,1624233600,3.00,3.0,3.0,3.0,3.0,일인용으로 쓰기에 적당합니다. 내구성도 좋아요,단품,2021,6,21


In [3]:
inter_recbole.columns = ['item_id', 'user_id', 'timestamp', 'star_avg', 'star_durability', 'star_design', 'star_cost', 'star_delivery', 'review', 'option', 'year', 'month', 'day']
inter_recbole.drop(['year', 'month', 'day'], axis=1, inplace=True)

In [4]:
inter_recbole.columns = ['item_id:token', 'user_id:token', 'timestamp:float', 'star_avg:float', 'star_durability:float', 'star_design:float', 'star_cost:float', 'star_delivery:float', 'review:token_seq', 'option:token_seq']
inter_recbole.to_csv("final.csv", index=False)

# item

In [7]:
side = pd.read_csv("side_info.csv")
side.head(2)

,item_id,title,brand,original_price,selling_price,wish_count,share_count,scrap_count,qa_count,review_count,...,is_selling,is_buyable,category0,category1,delivery_fee,delivery_fee_free_threshold,img_main,img_sub_list,option_names,max_option_price
0,265527,마르틴 원목 사이드보드 2colors,장미맨숀,938000,469000,9380,517,9380,245,198,...,True,True,가구,거실장·TV장,30000,0,https://image.ohou.se/image/central_crop/bucke...,['https://image.ohou.se/image/central_crop/buc...,"['색상:클래식화이트', '색상:램프블랙']",469000
1,456986,600ml 스텐 우유 스팀피쳐/눈금형,벨라쿠진,19800,9900,1118,31,1118,10,153,...,True,True,주방용품,커피·티용품,3000,10000,https://image.ohou.se/image/central_crop/bucke...,['https://image.ohou.se/image/central_crop/buc...,['선택:600ml 스텐 우유 스팀피쳐/눈금형'],9900


In [8]:
# side_recbole = side.drop(['is_selling', 'is_buyable', 'img_main', 'img_sub_list', 'option_names'], axis=1)
# side_recbole['is_sold_out'] = side_recbole['is_sold_out'].apply(int)
# side_recbole.columns = ['item_id:token', 'title:token_seq', 'brand:token', 'original_price:float', 'selling_price:float',
#        'wish_count:float', 'share_count:float', 'scrap_count:float', 'qa_count:float', 'review_count:float',
#        'review_avg:float', 'is_sold_out:float', 'category0:token', 'category1:token', 'delivery_fee:float', 'delivery_fee_free_threshold:float',
#        'max_option_price:float']
side_recbole = side.drop(['is_selling', 'is_buyable', 'img_main', 'img_sub_list', 'option_names', 'title'], axis=1)
side_recbole['is_sold_out'] = side_recbole['is_sold_out'].apply(int)
side_recbole.columns = ['item_id:token', 'brand:token', 'original_price:float', 'selling_price:float',
       'wish_count:float', 'share_count:float', 'scrap_count:float', 'qa_count:float', 'review_count:float',
       'review_avg:float', 'is_sold_out:float', 'category0:token', 'category1:token', 'delivery_fee:float', 'delivery_fee_free_threshold:float',
       'max_option_price:float']
side_recbole.head(2)

,item_id:token,brand:token,original_price:float,selling_price:float,wish_count:float,share_count:float,scrap_count:float,qa_count:float,review_count:float,review_avg:float,is_sold_out:float,category0:token,category1:token,delivery_fee:float,delivery_fee_free_threshold:float,max_option_price:float
0,265527,장미맨숀,938000,469000,9380,517,9380,245,198,4.560606,0,가구,거실장·TV장,30000,0,469000
1,456986,벨라쿠진,19800,9900,1118,31,1118,10,153,4.777778,0,주방용품,커피·티용품,3000,10000,9900


In [97]:
side_recbole.to_csv("final.csv", index=False)

# user